In [ ]:
# Libraries
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
from scipy.stats import poisson, chi2, chisquare, norm
import matplotlib.pyplot as plt
import seaborn as sns
import this

In [ ]:
# Versions
print("pandas version:", pd.__version__)
print("numpy version:", np.__version__)
#print("statsmodels version:", sm.__version__)

In [ ]:
# Directories & Files
os.listdir()

# Datasets directory
directory = "./datasets/"

In [ ]:
# Setting the 
datasets = {"five38": "spi_matches.csv", 
            "belgium": "belgium.csv",
            "champions": "champs.csv",
            "deductions": "deductions.csv",
            "eng_club_data": "england_club_data.csv",
            "eng_nonleague": "england_nonleague.csv",
            "england": "england.csv",
            "eng_playoffs": "englandplayoffs.csv",
            "fa_cup": "facup.csv",
            "france": "france.csv",
            "germany": "germany.csv",
            "germany2": "germany2.csv",
            "greece": "greece.csv",
            "holland": "holland.csv",
            "italy": "italy.csv",
            "league_cup_test": "leagucuptest.csv",
            "league_cup": "leaguecup.csv",
            "mls": "mls.csv",
            "mls_conference": "mlsconfs.csv",
            "portugal": "portugal.csv",
            "south_africa": "safrica.csv",
            "scotland": "scotland.csv",
            "turkey": "turkey.csv",
            "team_names": "teamnames.csv",
            "spain": "spain.csv"}

In [ ]:
# Creating the DataFrames
for name in datasets:
    print(name)
    globals()[name] = pd.read_csv(directory + datasets[name])
    
#dataframes = {f"{name}": globals()[name] = pd.read_csv(directory + datasets[name]) for name in datasets}

In [ ]:
# Cleaning five38
five38.head()

In [ ]:
# reducing five38 to date, league, team1 (Home Team), team2 (Away Team), score1 (Home Goals), 
# score 2 (Away Goals)

five_redux = five38[["date", "league", "team1", "team2", "score1", "score2"]]
five_redux

In [ ]:
# checking Series types
five_redux.dtypes

In [ ]:
# checking for nulls
print("Nulls:\n", five_redux.isnull().sum())
print()
print("Na's %:\n", five_redux.isna().mean().round(4) * 100)

In [ ]:
# defining a couple of functions to provide me info about the nulls
def null_cols(ds):
    """check whether the value in each field is missing (null) and return either 
    True or False for each field, totaling up the number of True values by column."""
    return ds.isnull().sum()

def na_cols(ds):
    """Does the same as null_cols, but returns the value as a percentage. 
    Useful to decide where to drop."""
    return ds.isna().mean().round(4) * 100

In [ ]:
# creating a dict to map df names to 
dataframes = {"five38_redux": five_redux, 
            "be": belgium,
            "ucl": champions,
            "duducts": deductions,
            "eng_club": eng_club_data,
            "eng_non_league": eng_nonleague,
            "eng": england,
            "england_playoffs": eng_playoffs,
            "cup_fa": fa_cup,
            "fr": france,
            "ger": germany,
            "ger2": germany2,
            "gr": greece,
            "ned": holland,
            "ita": italy,
            "l_cup_test": league_cup_test,
            "l_cup": league_cup,
            "usa": mls,
            "conference": mls_conference,
            "pt": portugal,
            "sa": south_africa,
            "sco": scotland,
            "trkey": turkey,
            "teamnames": team_names,
            "spa": spain}

In [ ]:
# checking if the df's have (or not) nulls in them
for name in dataframes:
    print(name, null_cols(dataframes[name]), "\n")

# five38_redux has on score1 and score 2
# ucl has on leg, HT, aet, pens, aethgoal, aetvgoal and tiewinner
# eng_club has on highest_div, col1, col2, short_name, three_letter_name, nicknames
# eng_non_league has on Date
# eng has on division
# england_playoffs has on aet, pen, Venue, attendance, neutral
# cup_fa has on Date, visitor, FT, hgoal, vgoal, tie, aet, pen, pens, hp, vp, Venue, \
# attendance, nonmatch, notes, neutral
# l_cup_test has on aet, pens, Venue, attendance, northsouth
# l_cup has on aet, pens, Venue, attendance, northsouth
# usa has on leg, hgoalaet, vgoalaet, hpen, vpen
# conference has on every col except team
# teamnames has on name_other and most_recent
# spa has on HT, group and notes

# be, duducts Season, fr, ger, ger2, ned, ita, pt, sa, sco, trkey,  don't

In [ ]:
# looking at nulls characteristics
five_redux[five_redux.isnull().any(axis = 1)]["date"].unique()

In [ ]:
# droping all nulls that are from today (26 Feb. 2020) onwards
# AFTER converting dtype to datetime 
five_redux["date"] = pd.to_datetime(five_redux["date"])#, format = "%d/%m/%Y")

In [ ]:
# checking if "date" was converted to datetime correctly
five_redux.dtypes

In [ ]:
# so NOW dropping the nulls from this date onwards (it had 3830 in score1 and score 2):
new_five38 = five_redux.loc[(five_redux["date"] < "2020-02-26")]

In [ ]:
# now it only has 10 in each column, so we can see them:
new_five38.isna().sum()

# entity 26630 was Postponed, 28235 was Suspended, 29616 was Postponed, 29624 was Postponed,
# 29884 was Postponed, and all the others where postponed due to the Corona Virus Outbreak
# in Italy, so I'll drop them to.
new_five38.loc[five_redux.isnull().any(axis = 1)]

In [ ]:
# droping
new_five38.dropna(inplace = True)

In [ ]:
# checking if they where droped
new_five38.isna().sum()

# comparing the original dataframe to the actual (from: 34010, to: 30180, which is: 3830. 
# so all should be well
print(five38.shape, new_five38.shape)

In [ ]:
# rechecking the head
new_five38.head()

In [ ]:
# chaging the names of the Series
new_five38.rename(columns = {"team1": "home", "team2": "away", 
                             "score1": "home_goals", "score2": "away_goals"}, inplace = True)

In [ ]:
# checking
new_five38

In [ ]:
# adding a new Series, "t_goals", which sums the home and away goals
new_five38["t_goals"] = new_five38["home_goals"] + new_five38["away_goals"]

In [ ]:
# checking
new_five38

In [ ]:
# Creating a "season" Series. This one maybe a bit trickier...
new_five38.loc[(five_redux["date"] > "2019-07-01") & (five_redux["date"] < "2020-07-01") &
               (five_redux["league"] != "Major League Soccer") | 
               (five_redux["league"] != "United Soccer League")]

In [ ]:
portugal

In [ ]:
# checking summary stats again
round(new_five38.describe(), 2)

In [ ]:
# checking the max goals
new_five38.loc[new_five38["t_goals"] == new_five38["t_goals"].max()]
new_five38.loc[new_five38["home_goals"] == new_five38["home_goals"].max()]
new_five38.loc[new_five38["away_goals"] == new_five38["away_goals"].max()]

In [ ]:
## EDA
new_five38.columns

In [ ]:
# looking at the distribuition of the data
# staging the figure area and arranging the plots
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize = (20, 20))

# preparing the first plot
ax1.hist(new_five38["home_goals"], bins = 10, edgecolor = "white")
ax1.set_title("home goals")

# the second plot
ax2.hist(new_five38["away_goals"], bins = 9, edgecolor = "white")
ax2.set_title("away goals")

# and the third
ax3.hist(new_five38["t_goals"], bins = 12, edgecolor = "white")
ax3.set_title("total goals")

# setting the bkg color to white (in case of export: https://stackoverflow.com/questions/4804005/matplotlib-figure-facecolor-background-color)
fig.set_facecolor("white")
plt.show()

In [ ]:
# looking at the distribuition of the data w/
# creating a Density plot

plt.figure(figsize = (16, 10))
sns.set(rc = {"axes.facecolor": "white", "figure.facecolor": "grey"})

p1 = sns.kdeplot(new_five38["home_goals"], shade = True, kernel = "tri", alpha = 0.20, 
                 bw = 1, color = "darkturquoise")
p1 = sns.kdeplot(new_five38["away_goals"], shade = True, kernel = "tri", alpha = 0.20, 
                 bw = 1, color = "sandybrown")
p1 = sns.kdeplot(new_five38["t_goals"], shade = True, kernel = "tri", alpha = 0.20, 
                 bw = 1, color = "mediumvioletred")

In [ ]:
# Inferential Statistics..?

In [ ]:
# Calculate a few first moments(??)

# Home Goals
home_mu = new_five38["home_goals"].mean()
home_mean, home_var, home_skew, home_kurt = poisson.stats(home_mu, moments = "mvsk")

# Away Goals
away_mu = new_five38["away_goals"].mean()
away_mean, away_var, away_skew, away_kurt = poisson.stats(away_mu, moments = "mvsk")

# Total Goals
t_mu = new_five38["t_goals"].mean()
t_mean, t_var, t_skew, t_kurt = poisson.stats(t_mu, moments = "mvsk")

In [ ]:
# Display the probability mass function (pmf):
fig, (axhome, axaway, axt) = plt.subplots(3, 1, figsize = (12, 16))

# Home Goals
home_x = np.arange(poisson.ppf(0.01, home_mu), poisson.ppf(0.99, home_mu))
axhome.plot(home_x, poisson.pmf(home_x, home_mu), "bo", ms = 8, 
            label = "Home Goals poisson pmf")
axhome.vlines(home_x, 0, poisson.pmf(home_x, home_mu), colors = "b", lw = 5, alpha = 0.5)

# Away Goals
away_x = np.arange(poisson.ppf(0.01, away_mu), poisson.ppf(0.99, away_mu))
axaway.plot(away_x, poisson.pmf(away_x, away_mu), "bo", ms = 8, 
            label = "Away Goals poisson pmf")
axaway.vlines(away_x, 0, poisson.pmf(away_x, away_mu), colors = "b", lw = 5, alpha = 0.5)

# Total Goals
t_x = np.arange(poisson.ppf(0.01, t_mu), poisson.ppf(0.99, t_mu))
axt.plot(t_x, poisson.pmf(t_x, t_mu), "bo", ms = 8, label = "Total Goals poisson pmf")
axt.vlines(t_x, 0, poisson.pmf(t_x, t_mu), colors = "b", lw = 5, alpha = 0.5)

In [ ]:
# Freeze the distribution and display the frozen pmf:
fig, (axhome, axaway, axt) = plt.subplots(3, 1, figsize = (12, 16))

# Home Goals
home_rv = poisson(home_mu)
axhome.vlines(home_x, 0, home_rv.pmf(home_x), colors = 'k' , linestyles = "-", lw = 1, 
          label = "Home Goals frozen pmf")
axhome.legend(loc = "best", frameon = False)

# Away Goals
away_rv = poisson(away_mu)
axaway.vlines(away_x, 0, away_rv.pmf(away_x), colors = 'k' , linestyles = "-", lw = 1, 
          label = "Away Goals frozen pmf")
axhome.legend(loc = "best", frameon = False)

# Total Goals
t_rv = poisson(t_mu)
axt.vlines(t_x, 0, t_rv.pmf(t_x), colors = 'k' , linestyles = "-", lw = 1, 
           label = "Total Goals frozen pmf")
axt.legend(loc = "best", frameon = False)

plt.show()

In [ ]:
#Check accuracy of cdf and ppf:
home_prob = poisson.cdf(home_x, home_mu)
away_prob = poisson.cdf(away_x, away_mu)
t_prob = poisson.cdf(t_x, t_mu)

# Return Results:
print(f"Home Goals Accuracy of cdf and ppf: {np.allclose(home_x, poisson.ppf(home_prob, home_mu))}\nAway Goals Accuracy of cdf and ppf: {np.allclose(away_x, poisson.ppf(away_prob, away_mu))}\nTotal Goals Accuracy of cdf and ppf: {np.allclose(t_x, poisson.ppf(t_prob, t_mu))}")

In [ ]:
# Create Poisson Distribution w/the mean of each col (away goals, home goals, and total goals);
# include events that haven't occured (13 goals, 14, 28... +inf), to create expected occurences;

# is the mean right?
home_expected_goals = poisson(mu = new_five38["home_goals"].mean())

x_axis = np.arange(0, len(new_five38["home_goals"].value_counts()))
plt.plot(x_axis, home_expected_goals.pmf(x_axis))

In [ ]:
home_expected_goals.cdf(5)

In [ ]:
# Create Poisson pmf w/value_counts of each occurence for each col, with 0 for + that 
# col.max() +1 to create observed occurences;
home_observed_goals = new_five38["home_goals"].value_counts(normalize = True, 
                                                            sort = False)

# TODO: Convert the abs freq of one to rel freq (or vice-versa for the other), so that they 
# can be compared;

# compare observed to expected to get the p-value and get the answer!
#from scipy.stats import kstest
observed_values = np.array(new_five38["home_goals"].value_counts().sort_index())
expected_values = [int(home_expected_goals.pmf(i) * len(new_five38["home_goals"])) 
                   for i in range(11)]

print("observed:", observed_values)
print("expected", expected_values)
print()
print("chi...........", chisquare(observed_values, expected_values))
#print(kstest(observed_values, home_expected_goals.cdf))

x_axis = np.arange(0, len(new_five38["home_goals"].value_counts()))
plt.plot(x_axis, home_observed_goals)
plt.plot(x_axis, home_expected_goals.pmf(x_axis))
#print(home_observed_goals)

In [ ]:
# Create Poisson Distribution w/the mean of each col (away goals, home goals, and total goals);
# include events that haven't occured (13 goals, 14, 28... +inf), to create expected occurences;

# is the mean right?
away_expected_goals = poisson(mu = new_five38["away_goals"].mean())

x_axis = np.arange(0, len(new_five38["away_goals"].value_counts()))
plt.plot(x_axis, away_expected_goals.pmf(x_axis))

In [ ]:
# Create Poisson pmf w/value_counts of each occurence for each col, with 0 for + that 
# col.max() +1 to create observed occurences;
away_observed_goals = new_five38["away_goals"].value_counts(normalize = True, 
                                                            sort = False)

# TODO: Convert the abs freq of one to rel freq (or vice-versa for the other), so that they 
# can be compared;

# compare observed to expected to get the p-value and get the answer!
observed_values = np.array(new_five38["away_goals"].value_counts().sort_index())
expected_values = [int(home_expected_goals.pmf(i) * len(new_five38["away_goals"])) 
                   for i in range(11)]

print("observed:", observed_values)
print("expected", expected_values)
print()
print("chi...........", chisquare(observed_values, expected_values))
#print(kstest(observed_values, home_expected_goals.cdf))

x_axis = np.arange(0, len(new_five38["home_goals"].value_counts()))
plt.plot(x_axis, home_observed_goals)
plt.plot(x_axis, home_expected_goals.pmf(x_axis))
#print(home_observed_goals)

# IGNORE

In [ ]:
# Chi2 test(s?) functions..?

def getBins(xmin, xmax, n_bins):
    """
    I copied from: 
    https://stats.stackexchange.com/questions/202617/implementing-chi-square-in-python-and-testing-on-scipys-poisson-and-norm-variat
    I actully have no idea what it's doing.
    """
    r = np.linspace(xmin, xmax, num = n_bins + 1, endpoint = True)
    r = r + 10 ** (-10) # including rightmost
    r[0] = r[0] - 2 * 10 ** (-10) # excluding xmin from (-Inf;xmin] bin
    return np.concatenate((np.array([float('-inf')]), r, np.array([float('inf')])))

# Calculates probabilities for each bin (a,b] within given cumulative distribution function 
def piCalcDecoratorNew(bins, *args):
    def real_piCalcDecorator(cdfFunc):
        def piCalc(*args):
            piA = np.zeros(len(bins) - 1)
            if len(args) == 1:
                args = args[0]
                piA[0] = cdfFunc(bins[1], args)  
                piA[-1] = 1 - cdfFunc(bins[-2], args)
                for i in range(1, len(bins) - 2):
                    piA[i] = cdfFunc(bins[i + 1], args) - cdfFunc(bins[i], args)
            else: #number of params >1
                piA[0] = cdfFunc(bins[1], *args)            
                piA[-1] = 1 - cdfFunc(bins[-2], *args)
                for i in range(1, len(bins) -2):
                    piA[i] = cdfFunc(bins[i + 1], *args) - cdfFunc(bins[i], *args)
            return piA  
        return piCalc
    return real_piCalcDecorator

# similar to scipy's chisquare()
def chi2statistic(obs = np.array([16, 18, 16, 14, 12, 12], dtype = 'float'), 
                  exp = np.array([16, 16, 16, 16, 16, 8], dtype = 'float')):
    temp = np.square(obs - exp, dtype = 'float')
    with np.errstate(divide = 'ignore', invalid = 'ignore'):
        temp = temp / exp
        temp[exp == 0] = 0
    return sum(temp)
    # like return chisquare(obs, exp)

def chi2test(df, x, alpha, cdfFunc, *args):
    N = len(x)
    xmin = min(x)
    xmax = max(x)
    bins = getBins(xmin, xmax, df - 2)
    print("Bins for histogram are ")
    print(bins)
    piCalc = piCalcDecoratorNew(bins, *args)(cdfFunc)
    piks = piCalc( *args)
    print("Expected probability to be in a bin")
    print(piks)
    a = piks * float(N)
    b = np.histogram(x, bins)[0]
    print("Observed probabilities for bins")
    print(b / float(N))
    print("Chi2 statistic is {0}".format(chi2statistic(b,a)))
    print("Critical value is {0}".format(chi2.ppf(alpha,df)))
    return (chi2statistic(b,a),chi2.ppf(alpha,df))

In [ ]:
# Testing on scipy's norm
alpha = 0.05
test_sequence = norm.rvs(loc = 10.0, scale = 2.0, size = 100000, random_state = 42)
print(chi2test(5, test_sequence, alpha, norm.cdf, 10.0, 2.0))

# Testing on scipy's poisson
alpha = 0.05
test_sequence = poisson.rvs(mu = 10, size = 100000, random_state = 42)
print(chi2test(5, test_sequence, alpha, poisson.cdf, 10.0))

# Testing on scipy's poisson with other parameters
alpha = 0.05
test_sequence = poisson.rvs(mu = 10, size = 300000, random_state = 42)

print(chi2test(max(test_sequence), test_sequence, alpha, poisson.cdf, 10.0))

In [ ]:
# Actually testing..?
# Testing on scipy's poisson
alpha = 0.05

# Home Goals
test_sequence = poisson.rvs(mu = home_mu, size = len(new_five38["home_goals"]), 
                            random_state = 42)

print("For Home Goals:")
print(chi2test(max(test_sequence), test_sequence, alpha, poisson.cdf, 10.0))
print()

# Away Goals
test_sequence = poisson.rvs(mu = away_mu, size = len(new_five38["away_goals"]), 
                            random_state = 42)

print("For Away Goals:")
print(chi2test(max(test_sequence), test_sequence, alpha, poisson.cdf, 10.0))
print()

# Total Goals
test_sequence = poisson.rvs(mu = t_mu, size = len(new_five38["t_goals"]), 
                            random_state = 42)

print("For Total Goals:")
print(chi2test(max(test_sequence), test_sequence, alpha, poisson.cdf, 10.0))
print()

In [ ]:
observed_values = np.array([18,21,16,7,15])
expected_values = np.array([22,19,44,8,16])

chisquare(observed_values, f_exp = expected_values)

In [ ]:
new_five38["home_goals"]